**Source**
kaggle: https://www.kaggle.com/c/payment-id-ndsc-2020

## Ratio (0.99982)

In [ ]:
import re
import time
import pandas as pd

from fuzzywuzzy import fuzz

In [ ]:
checkout = pd.read_csv('dataset/checkout.csv')
checkout.info()

In [ ]:
statement = pd.read_csv('dataset/bank_statement.csv')
statement.info()

In [ ]:
# hapus karakter yang tidak penting
allowed_char = re.compile(r'([^a-z])')
longer_space = re.compile(r'\s{2,}')

def preprocess_name(name):
    name = allowed_char.sub(' ', name)
    name = longer_space.sub(' ', name)
    return name

In [ ]:
# ganti nama kolom 
checkout.rename(columns={'ckt_amount': 'amount', 'ckt_id': 'id'}, inplace=True)
statement.rename(columns={'stmt_amount': 'amount', 'stmt_id': 'id'}, inplace=True)

# preprocess name 
checkout['buyer_name'] = checkout['buyer_name'].str.lower().map(preprocess_name)
statement['desc'] = statement['desc'].str.lower().map(preprocess_name)

statement.head()

In [ ]:
checkout_df = checkout.copy()
statement_df = statement.copy()
statement_df.shape

In [ ]:
checkout_df = checkout_df.reset_index(drop=True).set_index('id')

statement_df = statement_df.reset_index(drop=True).set_index('id')
statement_df.head()

In [ ]:
start_time = time.time()

# nilai unik dari amount
amount_ls = sorted(list(set(statement_df.amount)))
match_dict = {}
for amt in amount_ls:

    # ambil data yang memenuhi nilai amount tertentu dan ubah ke format dictionary
    cout_dict = dict(checkout_df[checkout_df['amount']==amt]['buyer_name'])
    stmt_dict = dict(statement_df[statement_df['amount']==amt]['desc'])

    # Kondisi-1 = nama pembeli ada di deskripsi
    for cout_id in cout_dict.copy():

        for stmt_id in stmt_dict.copy():
            # jika match maka simpan pasangan id nya 
            # dan hapus datanya dari dictionary
            if cout_dict[cout_id] in stmt_dict[stmt_id]:
                match_dict[cout_id] = stmt_id
                del cout_dict[cout_id]
                del stmt_dict[stmt_id]
                break

    # Kondisi-2 = nama pembeli dengan tidak memperhatikan urutan katanya
    for cout_id in cout_dict.copy():

        for stmt_id in stmt_dict.copy():
            # cek jika semua kata pada nama pembeli ada di deskripsi
            cout_word_ls = cout_dict[cout_id].split()
            if all(word in stmt_dict[stmt_id] for word in cout_word_ls):
                match_dict[cout_id] = stmt_id
                del cout_dict[cout_id]
                del stmt_dict[stmt_id]
                break

    # Kondisi-3 = matching algorithm menggunakan fuzzy string searching {ratio, partial_ratio, token_set_ratio}
    similarity_ls = []
    for cout_id, cout_name in cout_dict.items():

        for stmt_id, stmt_desc in stmt_dict.items():
            similarity_ls.append([cout_id, stmt_id, fuzz.ratio(cout_name, stmt_desc)])

    sorted_sim = sorted(similarity_ls, key=lambda x: x[-1], reverse=True)

    # Simpan pasangan id dengan nilai similarity tertinggi
    # dan hapus pasangan id lainnya yang mengandung id tersebut
    while(sorted_sim.copy()):
        cout_id = sorted_sim[0][0]
        stmt_id = sorted_sim[0][1]
        match_dict[cout_id] = stmt_id

        for data in sorted_sim.copy():
            if cout_id==data[0]:
                del sorted_sim[sorted_sim.index(data)]

        for data in sorted_sim.copy():
            if stmt_id==data[1]:
                del sorted_sim[sorted_sim.index(data)]
                
print(len(match_dict))
print(f'Time elapsed = {time.time() - start_time} seconds')

In [ ]:
checkout_id = list(match_dict.keys())
statement_id = list(match_dict.values())

submit = pd.DataFrame({'stmt_id': statement_id, 'ckt_id': checkout_id})
submit = submit.sort_values(by='stmt_id')
submit

In [ ]:
print(len(set(checkout_id)))
print(len(set(statement_id)))

In [ ]:
submit.to_csv('round1_ratio.csv', index=False)
print('masukk broo')

## Partial Ratio (0.99988)

In [ ]:
checkout_df = checkout.copy()
statement_df = statement.copy()
statement_df.shape

In [ ]:
checkout_df = checkout_df.reset_index(drop=True).set_index('id')

statement_df = statement_df.reset_index(drop=True).set_index('id')
statement_df.head()

In [ ]:
start_time = time.time()

# nilai unik dari amount
amount_ls = sorted(list(set(statement_df.amount)))
match_dict = {}
for amt in amount_ls:

    # ambil data yang memenuhi nilai amount tertentu dan ubah ke format dictionary
    cout_dict = dict(checkout_df[checkout_df['amount']==amt]['buyer_name'])
    stmt_dict = dict(statement_df[statement_df['amount']==amt]['desc'])

    # Kondisi-1 = nama pembeli ada di deskripsi
    for cout_id in cout_dict.copy():

        for stmt_id in stmt_dict.copy():
            # jika match maka simpan pasangan id nya 
            # dan hapus datanya dari dictionary
            if cout_dict[cout_id] in stmt_dict[stmt_id]:
                match_dict[cout_id] = stmt_id
                del cout_dict[cout_id]
                del stmt_dict[stmt_id]
                break

    # Kondisi-2 = nama pembeli dengan tidak memperhatikan urutan katanya
    for cout_id in cout_dict.copy():

        for stmt_id in stmt_dict.copy():
            # cek jika semua kata pada nama pembeli ada di deskripsi
            cout_word_ls = cout_dict[cout_id].split()
            if all(word in stmt_dict[stmt_id] for word in cout_word_ls):
                match_dict[cout_id] = stmt_id
                del cout_dict[cout_id]
                del stmt_dict[stmt_id]
                break

    # Kondisi-3 = matching algorithm menggunakan fuzzy string searching {ratio, partial_ratio, token_set_ratio}
    similarity_ls = []
    for cout_id, cout_name in cout_dict.items():

        for stmt_id, stmt_desc in stmt_dict.items():
            similarity_ls.append([cout_id, stmt_id, fuzz.partial_ratio(cout_name, stmt_desc)])

    sorted_sim = sorted(similarity_ls, key=lambda x: x[-1], reverse=True)

    # Simpan pasangan id dengan nilai similarity tertinggi
    # dan hapus pasangan id lainnya yang mengandung id tersebut
    while(sorted_sim.copy()):
        cout_id = sorted_sim[0][0]
        stmt_id = sorted_sim[0][1]
        match_dict[cout_id] = stmt_id

        for data in sorted_sim.copy():
            if cout_id==data[0]:
                del sorted_sim[sorted_sim.index(data)]

        for data in sorted_sim.copy():
            if stmt_id==data[1]:
                del sorted_sim[sorted_sim.index(data)]
                
print(len(match_dict))
print(f'Time elapsed = {time.time() - start_time} seconds')

In [ ]:
checkout_id = list(match_dict.keys())
statement_id = list(match_dict.values())

submit = pd.DataFrame({'stmt_id': statement_id, 'ckt_id': checkout_id})
submit = submit.sort_values(by='stmt_id')
submit

In [ ]:
submit['ckt_id'].nunique()

In [ ]:
submit.to_csv('round1_partial_ratio.csv', index=False)
print('masukk broo')

## Token Set Ratio (0.99994)

In [ ]:
checkout_df = checkout.copy()
statement_df = statement.copy()
statement_df.shape

In [ ]:
checkout_df = checkout_df.reset_index(drop=True).set_index('id')

statement_df = statement_df.reset_index(drop=True).set_index('id')
statement_df.head()

In [ ]:
start_time = time.time()

# nilai unik dari amount
amount_ls = sorted(list(set(statement_df.amount)))
match_dict = {}
for amt in amount_ls:

    # ambil data yang memenuhi nilai amount tertentu dan ubah ke format dictionary
    cout_dict = dict(checkout_df[checkout_df['amount']==amt]['buyer_name'])
    stmt_dict = dict(statement_df[statement_df['amount']==amt]['desc'])

    # Kondisi-1 = nama pembeli ada di deskripsi
    for cout_id in cout_dict.copy():

        for stmt_id in stmt_dict.copy():
            # jika match maka simpan pasangan id nya 
            # dan hapus datanya dari dictionary
            if cout_dict[cout_id] in stmt_dict[stmt_id]:
                match_dict[cout_id] = stmt_id
                del cout_dict[cout_id]
                del stmt_dict[stmt_id]
                break

    # Kondisi-2 = nama pembeli dengan tidak memperhatikan urutan katanya
    for cout_id in cout_dict.copy():

        for stmt_id in stmt_dict.copy():
            # cek jika semua kata pada nama pembeli ada di deskripsi
            cout_word_ls = cout_dict[cout_id].split()
            if all(word in stmt_dict[stmt_id] for word in cout_word_ls):
                match_dict[cout_id] = stmt_id
                del cout_dict[cout_id]
                del stmt_dict[stmt_id]
                break

    # Kondisi-3 = matching algorithm menggunakan fuzzy string searching {ratio, partial_ratio, token_set_ratio}
    similarity_ls = []
    for cout_id, cout_name in cout_dict.items():

        for stmt_id, stmt_desc in stmt_dict.items():
            similarity_ls.append([cout_id, stmt_id, fuzz.token_set_ratio(cout_name, stmt_desc)])

    sorted_sim = sorted(similarity_ls, key=lambda x: x[-1], reverse=True)

    # Simpan pasangan id dengan nilai similarity tertinggi
    # dan hapus pasangan id lainnya yang mengandung id tersebut
    while(sorted_sim.copy()):
        cout_id = sorted_sim[0][0]
        stmt_id = sorted_sim[0][1]
        match_dict[cout_id] = stmt_id

        for data in sorted_sim.copy():
            if cout_id==data[0]:
                del sorted_sim[sorted_sim.index(data)]

        for data in sorted_sim.copy():
            if stmt_id==data[1]:
                del sorted_sim[sorted_sim.index(data)]
                
print(len(match_dict))
print(f'Time elapsed = {time.time() - start_time} seconds')

In [ ]:
checkout_id = list(match_dict.keys())
statement_id = list(match_dict.values())

submit = pd.DataFrame({'stmt_id': statement_id, 'ckt_id': checkout_id})
submit = submit.sort_values(by='stmt_id')
submit

In [ ]:
submit.to_csv('round1_token_set_ratio.csv', index=False)
print('masukk broo')

## Ratio + Preprocessing (0.99994)

In [ ]:
checkout_df = checkout.copy()
statement_df = statement.copy()
statement_df.shape

In [ ]:
token_ls = []
for desc in statement_df['desc'].values:
    token_ls.extend(desc.split())
len(token_ls)

In [ ]:
token_occur = Counter(token_ls)
token_occur.most_common(100)

In [ ]:
# hapus karakter yang tidak penting
no_name = re.compile(r'(?<=\s)(instant|transfer|shopee|to a c|transferto|funds|atransfer|savings)(?=\s)')
longer_space = re.compile(r'\s{2,}')

def preprocess_description(desc):
    desc = no_name.sub('', desc)
    desc = longer_space.sub(' ', desc)
    return desc

In [ ]:
checkout_df = checkout_df.reset_index(drop=True).set_index('id')

statement_df['desc'] = statement_df['desc'].map(preprocess_description)
statement_df = statement_df.reset_index(drop=True).set_index('id')
statement_df.head()

In [ ]:
start_time = time.time()

# nilai unik dari amount
amount_ls = sorted(list(set(statement_df.amount)))
match_dict = {}
for amt in amount_ls:

    # ambil data yang memenuhi nilai amount tertentu dan ubah ke format dictionary
    cout_dict = dict(checkout_df[checkout_df['amount']==amt]['buyer_name'])
    stmt_dict = dict(statement_df[statement_df['amount']==amt]['desc'])

    # Kondisi-1 = nama pembeli ada di deskripsi
    for cout_id in cout_dict.copy():

        for stmt_id in stmt_dict.copy():
            # jika match maka simpan pasangan id nya 
            # dan hapus datanya dari dictionary
            if cout_dict[cout_id] in stmt_dict[stmt_id]:
                match_dict[cout_id] = stmt_id
                del cout_dict[cout_id]
                del stmt_dict[stmt_id]
                break

    # Kondisi-2 = nama pembeli dengan tidak memperhatikan urutan katanya
    for cout_id in cout_dict.copy():

        for stmt_id in stmt_dict.copy():
            # cek jika semua kata pada nama pembeli ada di deskripsi
            cout_word_ls = cout_dict[cout_id].split()
            if all(word in stmt_dict[stmt_id] for word in cout_word_ls):
                match_dict[cout_id] = stmt_id
                del cout_dict[cout_id]
                del stmt_dict[stmt_id]
                break

    # Kondisi-3 = matching algorithm menggunakan fuzzy string searching {ratio, partial_ratio, token_set_ratio}
    similarity_ls = []
    for cout_id, cout_name in cout_dict.items():

        for stmt_id, stmt_desc in stmt_dict.items():
            similarity_ls.append([cout_id, stmt_id, fuzz.ratio(cout_name, stmt_desc)])

    sorted_sim = sorted(similarity_ls, key=lambda x: x[-1], reverse=True)

    # Simpan pasangan id dengan nilai similarity tertinggi
    # dan hapus pasangan id lainnya yang mengandung id tersebut
    while(sorted_sim.copy()):
        cout_id = sorted_sim[0][0]
        stmt_id = sorted_sim[0][1]
        match_dict[cout_id] = stmt_id

        for data in sorted_sim.copy():
            if cout_id==data[0]:
                del sorted_sim[sorted_sim.index(data)]

        for data in sorted_sim.copy():
            if stmt_id==data[1]:
                del sorted_sim[sorted_sim.index(data)]
                
print(len(match_dict))
print(f'Time elapsed = {time.time() - start_time} seconds')

In [ ]:
print(f'Time elapsed = {time.time() - start_time} seconds')

checkout_id = list(match_dict.keys())
statement_id = list(match_dict.values())

submit = pd.DataFrame({'stmt_id': statement_id, 'ckt_id': checkout_id})
submit = submit.sort_values(by='stmt_id')
submit

In [ ]:
print(len(set(checkout_id)))
print(len(set(statement_id)))

In [ ]:
submit.to_csv('round1_ratio_prep.csv', index=False)

## Partial Ratio + Preprocessing (0.99982)

In [ ]:
checkout_df = checkout.copy()
statement_df = statement.copy()
statement_df.shape

In [ ]:
token_ls = []
for desc in statement_df['desc'].values:
    token_ls.extend(desc.split())
len(token_ls)

In [ ]:
token_occur = Counter(token_ls)
token_occur.most_common(100)

In [ ]:
# hapus karakter yang tidak penting
no_name = re.compile(r'(?<=\s)(instant|transfer|shopee|to a c|transferto|funds|atransfer|savings)(?=\s)')
longer_space = re.compile(r'\s{2,}')

def preprocess_description(desc):
    desc = no_name.sub('', desc)
    desc = longer_space.sub(' ', desc)
    return desc

In [ ]:
checkout_df = checkout_df.reset_index(drop=True).set_index('id')

statement_df['desc'] = statement_df['desc'].map(preprocess_description)
statement_df = statement_df.reset_index(drop=True).set_index('id')
statement_df.head()

In [ ]:
start_time = time.time()

# nilai unik dari amount
amount_ls = sorted(list(set(statement_df.amount)))
match_dict = {}
for amt in amount_ls:

    # ambil data yang memenuhi nilai amount tertentu dan ubah ke format dictionary
    cout_dict = dict(checkout_df[checkout_df['amount']==amt]['buyer_name'])
    stmt_dict = dict(statement_df[statement_df['amount']==amt]['desc'])

    # Kondisi-1 = nama pembeli ada di deskripsi
    for cout_id in cout_dict.copy():

        for stmt_id in stmt_dict.copy():
            # jika match maka simpan pasangan id nya 
            # dan hapus datanya dari dictionary
            if cout_dict[cout_id] in stmt_dict[stmt_id]:
                match_dict[cout_id] = stmt_id
                del cout_dict[cout_id]
                del stmt_dict[stmt_id]
                break

    # Kondisi-2 = nama pembeli dengan tidak memperhatikan urutan katanya
    for cout_id in cout_dict.copy():

        for stmt_id in stmt_dict.copy():
            # cek jika semua kata pada nama pembeli ada di deskripsi
            cout_word_ls = cout_dict[cout_id].split()
            if all(word in stmt_dict[stmt_id] for word in cout_word_ls):
                match_dict[cout_id] = stmt_id
                del cout_dict[cout_id]
                del stmt_dict[stmt_id]
                break

    # Kondisi-3 = matching algorithm menggunakan fuzzy string searching {ratio, partial_ratio, token_set_ratio}
    similarity_ls = []
    for cout_id, cout_name in cout_dict.items():

        for stmt_id, stmt_desc in stmt_dict.items():
            similarity_ls.append([cout_id, stmt_id, fuzz.partial_ratio(cout_name, stmt_desc)])

    sorted_sim = sorted(similarity_ls, key=lambda x: x[-1], reverse=True)

    # Simpan pasangan id dengan nilai similarity tertinggi
    # dan hapus pasangan id lainnya yang mengandung id tersebut
    while(sorted_sim.copy()):
        cout_id = sorted_sim[0][0]
        stmt_id = sorted_sim[0][1]
        match_dict[cout_id] = stmt_id

        for data in sorted_sim.copy():
            if cout_id==data[0]:
                del sorted_sim[sorted_sim.index(data)]

        for data in sorted_sim.copy():
            if stmt_id==data[1]:
                del sorted_sim[sorted_sim.index(data)]
                
print(len(match_dict))
print(f'Time elapsed = {time.time() - start_time} seconds')

In [ ]:
print(f'Time elapsed = {time.time() - start_time} seconds')

checkout_id = list(match_dict.keys())
statement_id = list(match_dict.values())

submit = pd.DataFrame({'stmt_id': statement_id, 'ckt_id': checkout_id})
submit = submit.sort_values(by='stmt_id')
submit

In [ ]:
print(len(set(checkout_id)))
print(len(set(statement_id)))

In [ ]:
submit.to_csv('round1_partial_ratio_prep.csv', index=False)

## Token Set Ratio + Preprocessing (0.99994)

In [ ]:
checkout_df = checkout.copy()
statement_df = statement.copy()
statement_df.shape

In [ ]:
token_ls = []
for desc in statement_df['desc'].values:
    token_ls.extend(desc.split())
len(token_ls)

In [ ]:
token_occur = Counter(token_ls)
token_occur.most_common(100)

In [ ]:
# hapus karakter yang tidak penting
no_name = re.compile(r'(?<=\s)(instant|transfer|shopee|to a c|transferto|funds|atransfer|savings)(?=\s)')
longer_space = re.compile(r'\s{2,}')

def preprocess_description(desc):
    desc = no_name.sub('', desc)
    desc = longer_space.sub(' ', desc)
    return desc

In [ ]:
checkout_df = checkout_df.reset_index(drop=True).set_index('id')

statement_df['desc'] = statement_df['desc'].map(preprocess_description)
statement_df = statement_df.reset_index(drop=True).set_index('id')
statement_df.head()

In [ ]:
start_time = time.time()

# nilai unik dari amount
amount_ls = sorted(list(set(statement_df.amount)))
match_dict = {}
for amt in amount_ls:

    # ambil data yang memenuhi nilai amount tertentu dan ubah ke format dictionary
    cout_dict = dict(checkout_df[checkout_df['amount']==amt]['buyer_name'])
    stmt_dict = dict(statement_df[statement_df['amount']==amt]['desc'])

    # Kondisi-1 = nama pembeli ada di deskripsi
    for cout_id in cout_dict.copy():

        for stmt_id in stmt_dict.copy():
            # jika match maka simpan pasangan id nya 
            # dan hapus datanya dari dictionary
            if cout_dict[cout_id] in stmt_dict[stmt_id]:
                match_dict[cout_id] = stmt_id
                del cout_dict[cout_id]
                del stmt_dict[stmt_id]
                break

    # Kondisi-2 = nama pembeli dengan tidak memperhatikan urutan katanya
    for cout_id in cout_dict.copy():

        for stmt_id in stmt_dict.copy():
            # cek jika semua kata pada nama pembeli ada di deskripsi
            cout_word_ls = cout_dict[cout_id].split()
            if all(word in stmt_dict[stmt_id] for word in cout_word_ls):
                match_dict[cout_id] = stmt_id
                del cout_dict[cout_id]
                del stmt_dict[stmt_id]
                break

    # Kondisi-3 = matching algorithm menggunakan fuzzy string searching {ratio, partial_ratio, token_set_ratio}
    similarity_ls = []
    for cout_id, cout_name in cout_dict.items():

        for stmt_id, stmt_desc in stmt_dict.items():
            similarity_ls.append([cout_id, stmt_id, fuzz.token_set_ratio(cout_name, stmt_desc)])

    sorted_sim = sorted(similarity_ls, key=lambda x: x[-1], reverse=True)

    # Simpan pasangan id dengan nilai similarity tertinggi
    # dan hapus pasangan id lainnya yang mengandung id tersebut
    while(sorted_sim.copy()):
        cout_id = sorted_sim[0][0]
        stmt_id = sorted_sim[0][1]
        match_dict[cout_id] = stmt_id

        for data in sorted_sim.copy():
            if cout_id==data[0]:
                del sorted_sim[sorted_sim.index(data)]

        for data in sorted_sim.copy():
            if stmt_id==data[1]:
                del sorted_sim[sorted_sim.index(data)]
                
print(len(match_dict))
print(f'Time elapsed = {time.time() - start_time} seconds')

In [ ]:
print(f'Time elapsed = {time.time() - start_time} seconds')

checkout_id = list(match_dict.keys())
statement_id = list(match_dict.values())

submit = pd.DataFrame({'stmt_id': statement_id, 'ckt_id': checkout_id})
submit = submit.sort_values(by='stmt_id')
submit

In [ ]:
print(len(set(checkout_id)))
print(len(set(statement_id)))

In [ ]:
submit.to_csv('round1_token_set_ratio_prep.csv', index=False)